<img src='https://www.met.no/om-oss/logo/_/image/73f29cde-219f-487b-809c-9cdd61032c78:2efc46ce776f5f5337c4b0156ae0cbaa3b6bf6fe/width-768/Met_RGB_Horisontal.jpg' width=200 align=right>
<img src='https://raw.githubusercontent.com/norkyst/norkyst-logo/refs/heads/main/png/horizontal_35_91_100.png' width=200 align=right>

# __UNDER CONSTRUCTION__

# Vertical cross-sections using Norkyust v3 data

This notebook will give tips and tricks on how to produce vertical cross-sections using data from the Norwegian ocean circulation model Norkyst v3.

__About cross-sections:__

...

__Python requirements__:

...